<div class="alert alert-block alert-success">
<h1><b>PROJET 8 - </b>Participez à la conception d'une voiture autonome</h1>
<h2>Notebook Scripts</h2></div>

<img src="future_vision_transport.png" width="250" height="250">

## Sommaire<hr>
[INTRODUCTION & PRESENTATION](#section_0)<br>
[A - IMPORTATIONS, DATA COLLECTION & INITIAL DATAFRAME](#section_0A)<br>
[B - EXPLORATORY DATA ANALYSIS](#section_0B)<br>
[C - TEXT PREPROCESSING](#section_0C)<br>
[D - WORD EMBEDDINGS](#section_0D)<br>
&emsp;[1 - Our own Word2Vec](#section_0D1)<br>
&emsp;[2 - Google's Word2Vec](#section_0D2)<br>
&emsp;[3 - GloVe word embeddings](#section_0D3)<br>
&emsp;[4 - FastText](#section_0D4)<br>
[E - DATA AND DATASET PREPARATION FOR ALGORITHMS](#section_0E)<br>
[F - PERFORMANCE METRICS DEFINITION](#section_0F)<br><hr>

<h2><div class="alert alert-block alert-success" id="section_0">
INTRODUCTION & PRESENTATION</div></h2>

<h3><div class="alert alert-block alert-info" id="section_0A">
<b>A - </b>IMPORTATIONS, DATA COLLECTION & INITIAL DATAFRAME</div></h3>

In [1]:
%matplotlib inline

# Importation of Python modules and methods.
import warnings

# Importation of Machine Learning libraries.
import matplotlib.pyplot as plt
import numpy as np
# import pandas as pd
# import pickle5 as pickle

# Importation of TensorFlow/Keras functions or classes.
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import (
    Bidirectional,
    Dense,
    Dropout,
    Embedding,
    LSTM,
)
from tensorflow.keras.models import Sequential, load_model

In [2]:
import os
from tensorflow.keras.preprocessing import image
import Augmentor
import tqdm

In [3]:
image_dir = 'datasets/images'
mask_dir = 'datasets/masks'
image_list = os.listdir(image_dir)
mask_list = os.listdir(mask_dir)
image_list.sort()
mask_list.sort()
print(f'Number of images: {len(image_list)}\nNumber of masks: {len(mask_list)}')

# sanity check
for i in range(len(image_list)):
    assert image_list[i] == mask_list[i]
print(image_list[2], mask_list[2])

Number of images: 6
Number of masks: 4


AssertionError: 

In [ ]:
p = Augmentor.Pipeline('datasets/images', output_directory='aug_dataset/images')

In [ ]:
p.ground_truth('datasets/masks')
p.rotate(probability=1, max_left_rotation=10, max_right_rotation=10)
p.flip_left_right(probability=0.5)
p.zoom_random(probability=0.5, percentage_area=0.8)
p.skew(probability=0.5, magnitude=0.5)
p.skew_tilt(probability=0.5, magnitude=0.5)
p.random_distortion(probability=0.5,grid_height=4, grid_width=4, magnitude=4)
p.shear(probability=0.5, max_shear_left=10, max_shear_right=10)
p.gaussian_distortion(probability=0.5, corner='bell', method='in', grid_height=4, grid_width=4, magnitude=4)
p.skew_top_bottom(probability=0.5, magnitude=.5)
p.skew_left_right(probability=0.5, magnitude=.5)
p.skew_corner(probability=0.5, magnitude=.5)
p.resize(probability=1,width=256, height=256)